Blueksy Data Extraction
===

In [1]:
import os
import time

import atproto
import dotenv

import pandas as pd
from tqdm import tqdm

In [2]:
dotenv.load_dotenv()

True

In [3]:
client = atproto.Client()
my_profile = client.login(os.environ["BLUESKY_USERNAME"], os.environ["BLUESKY_PASSWORD"])

In [5]:
my_profile.did

'did:plc:fi73esdhzesw3xdqtzfsmt3g'

In [6]:
response = client.app.bsky.graph.get_follows(params={"actor": my_profile.did, "limit": 100})

In [7]:
response.subject

ProfileView(did='did:plc:fi73esdhzesw3xdqtzfsmt3g', handle='zwlevonian.bsky.social', associated=ProfileAssociated(activity_subscription=ProfileAssociatedActivitySubscription(allow_subscriptions='followers', py_type='app.bsky.actor.defs#profileAssociatedActivitySubscription'), chat=ProfileAssociatedChat(allow_incoming='all', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=None, labeler=None, lists=None, starter_packs=None, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:fi73esdhzesw3xdqtzfsmt3g/bafkreibltowtb7wydjvuet5yesfkb5bx7tnt3skmhlijowfo65wnvdkdxe@jpeg', created_at='2024-02-06T18:35:24.863Z', description='Human–computer interaction researcher. PhD from University of Minnesota. Tacoma, WA. Mastodon: zwlevonian@hci.social', display_name='Zach Levonian', indexed_at='2025-08-14T17:28:19.099Z', labels=[], status=None, verification=None, viewer=ViewerState(activity_subscription=None, blocked_by=False, blocking=None, 

In [8]:
def get_all(get_function, params: dict, list_key: str, limit: int = 100):
    params = params.copy()
    params["limit"] = limit
    items = []
    cursor = None
    while True:
        params["cursor"] = cursor
        response = get_function(
            params=params
        )
        items.extend(getattr(response, list_key))
        if response.cursor is None:
            break
        cursor = response.cursor
    return items

def get_all_follows(client: atproto.Client, actor: str):
    return get_all(client.app.bsky.graph.get_follows, params={'actor': actor}, list_key="follows")

def get_all_followers(client: atproto.Client, actor: str):
    return get_all(client.app.bsky.graph.get_followers, params={'actor': actor}, list_key="followers")
    

In [11]:
follows = get_all_follows(client, profile.did)
len(follows)

1219

In [24]:
followers = get_all_followers(client, profile.did)
len(followers)

300

In [22]:
follows[0]

ProfileView(did='did:plc:2mkgbhbhqvappkkorf2bzyrp', handle='jasonschreier.bsky.social', associated=ProfileAssociated(activity_subscription=ProfileAssociatedActivitySubscription(allow_subscriptions='followers', py_type='app.bsky.actor.defs#profileAssociatedActivitySubscription'), chat=ProfileAssociatedChat(allow_incoming='following', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=None, labeler=None, lists=None, starter_packs=None, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:2mkgbhbhqvappkkorf2bzyrp/bafkreib2xnu5b2a3r3qrsva2loml45jiclpbsn7id4wywsimyo5xhjva7m@jpeg', created_at='2023-04-27T14:10:37.690Z', description='Reporter at Bloomberg | co-host of Triple Click | New York Times bestselling author of Play Nice + Press Reset + Blood, Sweat, and Pixels | jschreier@gmail.com', display_name='Jason Schreier', indexed_at='2025-04-17T13:30:38.973Z', labels=[], status=None, verification=VerificationState(trusted_verifie

In [28]:
follow_dids = {profile.did for profile in follows}
follower_dids = {profile.did for profile in followers}
mutuals = follow_dids & follower_dids
len(mutuals)

182

In [10]:
my_follows = get_all_follows(client, my_profile.did)
my_followers = get_all_followers(client, my_profile.did)
my_follow_dids = {profile.did for profile in my_follows}
my_follower_dids = {profile.did for profile in my_followers}
mutuals = my_follow_dids & my_follower_dids
ds = []
for mutual in tqdm(mutuals):
    follows = get_all_follows(client, mutual)
    followers = get_all_followers(client, mutual)
    follow_dids = {profile.did for profile in follows}
    follower_dids = {profile.did for profile in followers}
    their_mutuals = follow_dids & follower_dids
    ds.append({
        "mutual": mutual,
        "n_follows": len(follows),
        "n_followers": len(followers),
        "n_mutuals": len(their_mutuals),
    })
    time.sleep(60)  # laziest possible way to deal with the rate limit
df = pd.DataFrame(ds)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 182/182 [3:14:49<00:00, 64.23s/it]


In [14]:
len(my_follow_dids & my_follower_dids), df.n_mutuals.mean(), df.n_mutuals.median()

(182, np.float64(663.1593406593406), np.float64(216.5))